In [14]:
import joblib
import numpy as np
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

# Load model dan vectorizer
vectorizer = joblib.load('vectorizer.pkl')
model_nlp = load_model('model_nlp.h5')

# Muat tokenizer yang telah dilatih sebelumnya
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

logistic_model = joblib.load('logistic_model.pkl')
model_mlp = joblib.load('mlp_model.pkl')

# Mapping manual label
mapping_label = {0: "negatif", 1: "netral", 2: "positif"}

# Kalimat baru untuk dites
kalimat = [
    "tiap main rank pasti aja ada player beban/males",
    "grafik sudah oke, efek visual skill dalam pertandingan sudah mantap, skin2 hero keren",
    "mohon di perhatikan lagi ,di perbaiki,jangan perbarui skin trus"
]

# Tokenisasi dan padding (untuk model LSTM)
sekuens = tokenizer.texts_to_sequences(kalimat)
kalimat_pad = pad_sequences(sekuens, maxlen=1000)  # Sesuaikan dengan panjang input saat training

# Prediksi menggunakan model LSTM
prediksi_nlp = model_nlp.predict(kalimat_pad)

# Prediksi menggunakan MLP dan Logistic Regression (untuk vectorizer)
kalimat_vec = vectorizer.transform(kalimat)
prediksi_mlp = model_mlp.predict(kalimat_vec)
prediksi_logreg = logistic_model.predict(kalimat_vec)

# Fungsi untuk mengambil argmax jika output berupa probabilitas
def get_pred_label(pred):
    return np.argmax(pred, axis=1) if len(pred.shape) > 1 and pred.shape[1] > 1 else pred

# Ambil label prediksi untuk masing-masing model
label_mlp = [mapping_label[i] for i in get_pred_label(prediksi_mlp)]
label_logreg = [mapping_label[i] for i in get_pred_label(prediksi_logreg)]
label_nlp = [mapping_label[i] for i in get_pred_label(prediksi_nlp)]


# Cetak hasil prediksi untuk setiap kalimat
for i, kalimat_item in enumerate(kalimat):
    print(f"Kalimat: {kalimat_item}")
    print(f"Prediksi Sentimen (LSTM): {label_nlp[i]}")  # LSTM model
    print(f"Prediksi Sentimen (MLP): {label_mlp[i]}")
    print(f"Prediksi Sentimen (Logistic Regression): {label_logreg[i]}")
    print("-" * 50)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
Kalimat: tiap main rank pasti aja ada player beban/males
Prediksi Sentimen (LSTM): negatif
Prediksi Sentimen (MLP): negatif
Prediksi Sentimen (Logistic Regression): negatif
--------------------------------------------------
Kalimat: grafik sudah oke, efek visual skill dalam pertandingan sudah mantap, skin2 hero keren
Prediksi Sentimen (LSTM): positif
Prediksi Sentimen (MLP): positif
Prediksi Sentimen (Logistic Regression): positif
--------------------------------------------------
Kalimat: mohon di perhatikan lagi ,di perbaiki,jangan perbarui skin trus
Prediksi Sentimen (LSTM): positif
Prediksi Sentimen (MLP): negatif
Prediksi Sentimen (Logistic Regression): negatif
--------------------------------------------------
